In [52]:
#Путь к файлу
pathToFile = 'E:\\загрузки'
fileName = 'predict.csv'

In [53]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as lr
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [54]:
#Чтение данных в формате csv
os.chdir(pathToFile)
data = pd.read_csv(fileName, delimiter = ',', header=0)

In [55]:
#Первичная обработка данных

#Проверка на пустые столбцы
notNullColumns = [col for col in data.columns if data.loc[:, col].notna().any()]

#Оставляем только столбцы с данными
data = data[notNullColumns]

#Убираем пустые значения в строках
data = data.dropna()

#Получаем корреляционную матрицу для проверки избыточности информации
corrMatix = data.drop(['c5'], axis=1).corr()

#Так как матрица симметрична оставляем только половину для удобства работы
j=0
for col in corrMatix.columns:
    corrMatix[col][j:] = 0
    j += 1

#Получаем имена столбцов, корреляция которых выше 0,9
corrColumns = [col for col in corrMatix.columns if len(corrMatix.loc[corrMatix[col] > 0.9, col]) > 0]

#Удаляем зависимые столбцы
data = data.drop(corrColumns, axis=1)

In [56]:
#Разделение выборки на предикторы (predictors) и отклик (response)
resp = data['c5']
pred = data.drop(['c5'], axis=1)

#Определяем число для разбиение выборки 30 на 70
num = int(len(data)*0.3)

#Разделение выборки на тестовую (test) и обучающую (training)
predTest,predTrn = pred[:num], pred[num:]
respTest,respTrn = resp[:num], resp[num:]

In [57]:
#Создаем Random Forest Model с 1000 деревьев решений 
rf = RandomForestRegressor(n_estimators = 1000, random_state = 100)

#Обучаем Random Forest model на тренировочных данных
rf.fit(predTrn, respTrn);

#Получаем отклик от тестовых данных
predictions = rf.predict(predTest)

In [58]:
#Вычисляем значение R^2
r2 = r2_score(respTest,predictions)
print('R^2: ', round(r2, 2))

#Вычисляем mean squared error
mse = mean_squared_error(respTest,predictions)
print('MSE: ', round(mse, 2))


R^2:  0.23
MSE:  48.44


In [63]:
import xgboost
model = xgboost.XGBClassifier()
model.fit(predTrn, respTrn)
y_pred = model.predict(predTest)

r2 = r2_score(respTest,y_pred)
print('R^2: ', round(r2, 2))

ModuleNotFoundError: No module named 'xgboost'